# Advanced Python

1. Iterator
   1. List comprehension
   2. Generator
   3. Generator expression
2. Stack frame
   1. `frame` object
3. Customizing module import with `sys.meta_path`
4. Descriptor
   1. Keep data on the instance
5. Metaclass
6. Type introspection and abstract base class (abc)
   1. Method resolution order (mro)
   2. Abstract base class (abc)
   3. Abstract method

In [1]:
import sys
import os
import pprint

# Iterator

While processing data in memory, they are iterated one by one.  Assume we have 10 elements in a list.

In [2]:
data = list(range(10))
print(data, type(data))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9] <class 'list'>


Python uses the [iterator protocol](https://docs.python.org/3/library/stdtypes.html#iterator-types) to get one element a time:

In [3]:
class ListIterator:

    def __init__(self, data):
        self.data = data
        self.it = None

    def __iter__(self):
        return self

    def __next__(self):
        if None is self.it:
            self.it = 0
        elif self.it >= len(self.data)-1:
            raise StopIteration
        else:
            self.it += 1
        return self.data[self.it]

The `for ... in ...` construct applies to the iterator object.  Every time the construct needs the next element, `ListIterator.__next__()` is called:

In [4]:
list_iterator = ListIterator(data)
print(list_iterator)
print(dir(list_iterator))
for i in list_iterator:
    print(i)

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'data', 'it']
0
1
2
3
4
5
6
7
8
9


Of course, you don't really need to write your own `ListIterator` for iterating a list, because Python builds in an iterator already:

In [5]:
list_iterator2 = iter(data)
print(list_iterator2)
print(dir(list_iterator2))
for i in list_iterator2:
    print(i)

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__length_hint__', '__lt__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__']
0
1
2
3
4
5
6
7
8
9


The built-in iterator is created by calling the `__iter__()` method on the container object (`iter()` simply does it for you):

In [6]:
list_iterator3 = data.__iter__()
print(list_iterator3)
for i in list_iterator3:
    print(i)

0
1
2
3
4
5
6
7
8
9


And the `for ... in ...` construct actually knows about the iterator protocol:

In [7]:
for i in data:
    print(i)

0
1
2
3
4
5
6
7
8
9


## List comprehension

List comprehension is the construct `[... for ... in ...]`.  Python borrowed the syntax of list comprehension from other languages, e.g., Haskell, and it follows the iterator protocol.  It is very convenient.  For example, the above `for` loop can be replaced by a one-liner:

In [8]:
print("\n".join([str(i) for i in data]))

0
1
2
3
4
5
6
7
8
9


## Generator

In [9]:
def list_generator(input_data):
    for i in input_data:
        yield i

generator = list_generator(data)
print(generator)
print(dir(generator))
for i in list_generator(data):
    print(i)

<generator object list_generator at 0x10914d250>
['__class__', '__del__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__name__', '__ne__', '__new__', '__next__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'close', 'gi_code', 'gi_frame', 'gi_running', 'gi_yieldfrom', 'send', 'throw']
0
1
2
3
4
5
6
7
8
9


## Generator expression

A more convenient way of creating a generator is to use the generator expression `(... for ... in ...)`.  Note this looks like the list comprehension `[... for ... in ...]`, but uses parentheses to replace the brackets.

In [10]:
generator2 = (i for i in data)
print(generator2)
print(dir(generator2))
for i in generator2:
    print(i)

<generator object <genexpr> at 0x10914dc50>
['__class__', '__del__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__name__', '__ne__', '__new__', '__next__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'close', 'gi_code', 'gi_frame', 'gi_running', 'gi_yieldfrom', 'send', 'throw']
0
1
2
3
4
5
6
7
8
9


By using the generator expression, the data printing one-liner can drop the brackets:

In [11]:
print("\n".join(str(i) for i in data))
# Compare the the list comprehension:
# print("\n".join( [ str(i) for i in data ] ))

0
1
2
3
4
5
6
7
8
9


# Python stack frame

(C)Python uses a stack-based interpreter.  We are allowed to peek all the previous stack frames:

In [12]:
import traceback

traceback.print_stack()

  File "/Users/yungyuc/hack/usr/opt37_190418/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/yungyuc/hack/usr/opt37_190418/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/yungyuc/hack/usr/opt37_190418/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/yungyuc/hack/usr/opt37_190418/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/yungyuc/hack/usr/opt37_190418/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/Users/yungyuc/hack/usr/opt37_190418/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "/Users/yungyuc/hack/usr/opt37_190418/lib/python3.7/asyncio/base_events.py", line 534, in run_forever
    self._run_once()
  File "/Users/yungyu

## `frame` object

We can get the `frame` object of the current stack frame using [`inspect.currentframe()`](https://docs.python.org/3/library/inspect.html#inspect.currentframe):

In [13]:
import inspect

f = inspect.currentframe()

A `frame` object has the following attributes:

* Namespace:
  * `f_builtins`: builtin namespace seen by this frame
  * `f_globals`: global namespace seen by this frame
  * `f_locals`: local namespace seen by this frame
* Other:
  * `f_back`: next outer frame object (this frame's caller)
  * `f_code`: code object being executed in this frame
  * `f_lasti`: index of last attempted instruction in bytecode
  * `f_lineno`: current line number in Python source code

In [14]:
print([k for k in dir(f) if not k.startswith('__')])

['clear', 'f_back', 'f_builtins', 'f_code', 'f_globals', 'f_lasti', 'f_lineno', 'f_locals', 'f_trace', 'f_trace_lines', 'f_trace_opcodes']


We can learn many things about the frame in the object.  For example, we can take a look in the builtin namespace:

In [15]:
print(f.f_builtins.keys())

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__build_class__', '__import__', 'abs', 'all', 'any', 'ascii', 'bin', 'breakpoint', 'callable', 'chr', 'compile', 'delattr', 'dir', 'divmod', 'eval', 'exec', 'format', 'getattr', 'globals', 'hasattr', 'hash', 'hex', 'id', 'input', 'isinstance', 'issubclass', 'iter', 'len', 'locals', 'max', 'min', 'next', 'oct', 'ord', 'pow', 'print', 'repr', 'round', 'setattr', 'sorted', 'sum', 'vars', 'None', 'Ellipsis', 'NotImplemented', 'False', 'True', 'bool', 'memoryview', 'bytearray', 'bytes', 'classmethod', 'complex', 'dict', 'enumerate', 'filter', 'float', 'frozenset', 'property', 'int', 'list', 'map', 'object', 'range', 'reversed', 'set', 'slice', 'staticmethod', 'str', 'super', 'tuple', 'type', 'zip', '__debug__', 'BaseException', 'Exception', 'TypeError', 'StopAsyncIteration', 'StopIteration', 'GeneratorExit', 'SystemExit', 'KeyboardInterrupt', 'ImportError', 'ModuleNotFoundError', 'OSError', 'EnvironmentError', 'IOEr

A mysterious `code` object:

In [16]:
print(f.f_code)

<code object <module> at 0x1092828a0, file "<ipython-input-13-dac680851f0c>", line 3>


Because a `frame` object holds everything a construct uses, after finishing using the `frame` object, make sure to break the reference to it.  If we don't do it, it may take long time for the interpreter to break the reference for you.

In [17]:
f.clear()
del f

In [18]:
for it, fi in enumerate(inspect.stack()):
    sys.stdout.write('frame #{}:\n  {}\n\n'.format(it, fi))

frame #0:
  FrameInfo(frame=<frame at 0x7fd994005500, file '<ipython-input-18-31f84b6669ed>', line 2, code <module>>, filename='<ipython-input-18-31f84b6669ed>', lineno=1, function='<module>', code_context=['for it, fi in enumerate(inspect.stack()):\n'], index=0)

frame #1:
  FrameInfo(frame=<frame at 0x7fd992548870, file '/Users/yungyuc/hack/usr/opt37_190418/lib/python3.7/site-packages/IPython/core/interactiveshell.py', line 3296, code run_code>, filename='/Users/yungyuc/hack/usr/opt37_190418/lib/python3.7/site-packages/IPython/core/interactiveshell.py', lineno=3296, function='run_code', code_context=['                    exec(code_obj, self.user_global_ns, self.user_ns)\n'], index=0)

frame #2:
  FrameInfo(frame=<frame at 0x7fd99450fcc0, file '/Users/yungyuc/hack/usr/opt37_190418/lib/python3.7/site-packages/IPython/core/interactiveshell.py', line 3214, code run_ast_nodes>, filename='/Users/yungyuc/hack/usr/opt37_190418/lib/python3.7/site-packages/IPython/core/interactiveshell.py', li

```python
#!/usr/bin/env python3

import sys
import inspect

def main():
    for it, fi in enumerate(inspect.stack()):
        sys.stdout.write('frame #{}:\n  {}\n\n'.format(it, fi))

if __name__ == '__main__':
    main()
```

In [19]:
!./showframe.py

frame #0:
  FrameInfo(frame=<frame at 0x7f849b635800, file './showframe.py', line 8, code main>, filename='./showframe.py', lineno=7, function='main', code_context=['    for it, fi in enumerate(inspect.stack()):\n'], index=0)

frame #1:
  FrameInfo(frame=<frame at 0x1088fa450, file './showframe.py', line 11, code <module>>, filename='./showframe.py', lineno=11, function='<module>', code_context=['    main()\n'], index=0)



# Customizing module import with `sys.meta_path`

Python [`importlib`](https://docs.python.org/3/library/importlib.html) allows high degree of freedom in customizing module import flow.  Here I will use an example to load a module, `onemod`, locating in an alternate directory, `altdir/`, and ask Python to load it from the non-standard location.

In [20]:
# Bookkeeping code: keep the original meta_path.
old_meta_path = sys.meta_path[:]
#sys.meta_path = old_meta_path[:-1]

`importlib` provides many facilities.  The theme in this example is [`sys.meta_path`](https://docs.python.org/3/library/sys.html#sys.meta_path).  It defines a list of ['MetaPathFinder'](https://docs.python.org/3/library/importlib.html#importlib.abc.MetaPathFinder) objects for customizing the import process.

In [21]:
sys.meta_path = old_meta_path
print(sys.meta_path)

[<class '_frozen_importlib.BuiltinImporter'>, <class '_frozen_importlib.FrozenImporter'>, <class '_frozen_importlib_external.PathFinder'>, <six._SixMetaPathImporter object at 0x106494a50>]


At this point, the `onemod` cannot be imported, because `altdir/` is not in `sys.path`:

In [22]:
try:
    import onemod
except ModuleNotFoundError as e:
    traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-22-d88007362a9b>", line 2, in <module>
    import onemod
ModuleNotFoundError: No module named 'onemod'


In a normal Python course, you will be asked to modify `sys.path` to include `altdir/` for correctly import `onemod`.  That is uninteresting, so we will use `MetaPathFinder`.  Here we subclass the abstract base class (ABC) and override the `find_spec()` method, to tell it to load the `onemod` module at the place we specify.

For our path finder to work, we need to properly set up a [`ModuleSpec`](https://docs.python.org/3/library/importlib.html#importlib.machinery.ModuleSpec), and create a [`SourceFileLoader`](https://docs.python.org/3/library/importlib.html#importlib.machinery.SourceFileLoader) object for it.

In [23]:
import importlib.abc
import importlib.machinery

class MyMetaPathFinder(importlib.abc.MetaPathFinder):
    def find_spec(self, fullname, path, target=None):
        if fullname == 'onemod':
            print('DEBUG: fullname: {} , path: {} , target: {}'.format(fullname, path, target))
            fpath = os.path.abspath('altdir/onemod.py')
            loader = importlib.machinery.SourceFileLoader('onemod', fpath)
            return importlib.machinery.ModuleSpec(fullname, loader, origin=fpath)
        else:
            return None

sys.meta_path = old_meta_path + [MyMetaPathFinder()]
print(sys.meta_path)

[<class '_frozen_importlib.BuiltinImporter'>, <class '_frozen_importlib.FrozenImporter'>, <class '_frozen_importlib_external.PathFinder'>, <six._SixMetaPathImporter object at 0x106494a50>, <__main__.MyMetaPathFinder object at 0x10917f950>]


It only deals with `onemod`.  To test it, ask it to load a module that does not exist:

In [24]:
try:
    import one_non_existing_module
except ModuleNotFoundError as e:
    traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-24-b957d42d7042>", line 2, in <module>
    import one_non_existing_module
ModuleNotFoundError: No module named 'one_non_existing_module'


But after the meta path finder is inserted into meta path, `onemod` can be loaded:

In [25]:
import onemod
print("show content in onemod module:", onemod.content)

DEBUG: fullname: onemod , path: None , target: None
show content in onemod module: string in onemod


See the module we loaded.  Compare it with a 'normal module'.

In [26]:
import re
print('onemod:', onemod)
print('re:', re)

onemod: <module 'onemod' (/Users/yungyuc/hack/code/nsd/notebook/20sp_nctu/14_advpy/altdir/onemod.py)>
re: <module 're' from '/Users/yungyuc/hack/usr/opt37_190418/lib/python3.7/re.py'>


And the module objects have an important field `__spec__`, which is the `ModuleSpec` we created:

In [27]:
print('onemod.__spec__:', onemod.__spec__)
print('re.__spec__:', re.__spec__)

onemod.__spec__: ModuleSpec(name='onemod', loader=<_frozen_importlib_external.SourceFileLoader object at 0x109182410>, origin='/Users/yungyuc/hack/code/nsd/notebook/20sp_nctu/14_advpy/altdir/onemod.py')
re.__spec__: ModuleSpec(name='re', loader=<_frozen_importlib_external.SourceFileLoader object at 0x1061fa290>, origin='/Users/yungyuc/hack/usr/opt37_190418/lib/python3.7/re.py')


In [28]:
# Bookkeeping code.
sys.modules.pop('onemod', None)

<module 'onemod' (/Users/yungyuc/hack/code/nsd/notebook/20sp_nctu/14_advpy/altdir/onemod.py)>

# Descriptor

The [descriptor protocol](https://docs.python.org/3/howto/descriptor.html) allows us to route attribute access to anywhere outside the class.

In [29]:
class ClsAccessor:
    """Routing access to all instance attributes to the descriptor object."""

    def __init__(self, name):
        self._name = name
        self._val = None

    def __get__(self, obj, objtype):
        print('On object {} , retrieve: {}'.format(obj, self._name))
        return self._val

    def __set__(self, obj, val):
        print('On object {} , update: {}'.format(obj, self._name))
        self._val = val

class MyClass:
    x = ClsAccessor('x')

o = MyClass()

See the message printed while getting the attribute 'x':

In [30]:
print(o.x)

On object <__main__.MyClass object at 0x1092bad90> , retrieve: x
None


Setting the attribute also shows a message:

In [31]:
o.x = 10
print(o.x)

On object <__main__.MyClass object at 0x1092bad90> , update: x
On object <__main__.MyClass object at 0x1092bad90> , retrieve: x
10


Because the attribute value is kept in the descriptor, and the descriptor is kept in the 'class' object, attributes of all instances of `MyClass` share the same value.

In [32]:
o2 = MyClass()
print(o2.x) # Not None!

On object <__main__.MyClass object at 0x10916b8d0> , retrieve: x
10


## Keep data on the instance

Having all instances sharing the attribute value isn't always desirable.  Descriptor protocol allows to bind the values to the instance too.

In [33]:
class InsAccessor:
    """Routing access to all instance attributes to alternate names on the instance."""

    def __init__(self, name):
        self._name = name

    def __get__(self, obj, objtype):
        print('On object {} , retrieve: {}'.format(obj, self._name))
        varname = '_acs' + self._name
        if not hasattr(obj, varname):
            setattr(obj, varname, None)
        return getattr(obj, varname)

    def __set__(self, obj, val):
        print('On object {} , update: {}'.format(obj, self._name))
        varname = '_acs' + self._name
        return setattr(obj, varname, val)

class MyClass2:
    x = InsAccessor('x')

mo = MyClass2()

Create an instance to test the descriptor.

In [34]:
print(mo.x)

On object <__main__.MyClass2 object at 0x1092ba450> , retrieve: x
None


In [35]:
mo.x = 10
print(mo.x)

On object <__main__.MyClass2 object at 0x1092ba450> , update: x
On object <__main__.MyClass2 object at 0x1092ba450> , retrieve: x
10


In a new instance, the value uses the initial value:

In [36]:
mo2 = MyClass2()
print(mo2.x)

On object <__main__.MyClass2 object at 0x1092ba8d0> , retrieve: x
None


# Metaclass

Python class is also an object.

In [37]:
class ClassIsObject:
    pass

print(ClassIsObject)
print(ClassIsObject.__dict__)

<class '__main__.ClassIsObject'>
{'__module__': '__main__', '__dict__': <attribute '__dict__' of 'ClassIsObject' objects>, '__weakref__': <attribute '__weakref__' of 'ClassIsObject' objects>, '__doc__': None}


[Metaclass](https://docs.python.org/3/reference/datamodel.html#metaclasses) allows programmers to customize class creation.

In [38]:
class AutoAccessor:
    """Routing access to all instance attributes to alternate names on the instance."""

    def __init__(self):
        self.name = None

    def __get__(self, obj, objtype):
        print('On object {} , retrieve: {}'.format(obj, self.name))
        varname = '_acs' + self.name
        if not hasattr(obj, varname):
            setattr(obj, varname, None)
        return getattr(obj, varname)

    def __set__(self, obj, val):
        print('On object {} , update: {}'.format(obj, self.name))
        varname = '_acs' + self.name
        return setattr(obj, varname, val)

class AutoAccessorMeta(type):

    def __new__(cls, name, bases, namespace):

        print('DEBUG before names:', name)
        print('DEBUG before bases:', bases)
        print('DEBUG before namespace:', namespace)

        for k, v in namespace.items():
            if isinstance(v, AutoAccessor):
                v.name = k

        # Create the class object for MyAutoClass.
        newcls = super(AutoAccessorMeta, cls).__new__(cls, name, bases, namespace)

        print('DEBUG after names:', name)
        print('DEBUG after bases:', bases)
        print('DEBUG after namespace:', namespace)

        return newcls

We will use the descriptor to test the metaclass.  The new descriptor class `AutoAccessor` doesn't take the attribute name in the constructor.  Instead, `AutoAccessorMeta` assigns the correct attribute name.

In [39]:
class MyAutoClass(metaclass=AutoAccessorMeta):
    x = AutoAccessor()  # Note: no name is given.

ao = MyAutoClass()

DEBUG before names: MyAutoClass
DEBUG before bases: ()
DEBUG before namespace: {'__module__': '__main__', '__qualname__': 'MyAutoClass', 'x': <__main__.AutoAccessor object at 0x106e88b10>}
DEBUG after names: MyAutoClass
DEBUG after bases: ()
DEBUG after namespace: {'__module__': '__main__', '__qualname__': 'MyAutoClass', 'x': <__main__.AutoAccessor object at 0x106e88b10>}


In [40]:
print(ao.x)

On object <__main__.MyAutoClass object at 0x1092e1450> , retrieve: x
None


In [41]:
ao.x = 20
print(ao.x)

On object <__main__.MyAutoClass object at 0x1092e1450> , update: x
On object <__main__.MyAutoClass object at 0x1092e1450> , retrieve: x
20


# Type introspection and abstract base class (abc)

In [42]:
class MyBaseClass:
    pass

class MyDerivedClass(MyBaseClass):
    pass

base = MyBaseClass()
derived = MyDerivedClass()

In [43]:
print('base {} MyBaseClass'.format('is' if isinstance(base, MyBaseClass) else 'is not'))

base is MyBaseClass


In [44]:
print('base {} MyDerivedClass'.format('is' if isinstance(base, MyDerivedClass) else 'is not'))

base is not MyDerivedClass


In [45]:
print('derived {} MyBaseClass'.format('is' if isinstance(derived, MyBaseClass) else 'is not'))

derived is MyBaseClass


In [46]:
print('derived {} MyDerivedClass'.format('is' if isinstance(derived, MyDerivedClass) else 'is not'))

derived is MyDerivedClass


## Method resolution order (mro)

Python uses the "C3" algorithm to determine the [method resolution order](https://www.python.org/download/releases/2.3/mro/) [1].

In [47]:
class A:
    def process(self):
        print('A process()')

class B(A):
    def process(self):
        print('B process()')
        super(B, self).process()

class C(A):
    def process(self):
        print('C process()')
        super(C, self).process()

class D(B, C):
    pass

print(D.__mro__)

(<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>)


In [48]:
obj = D()
obj.process()

B process()
C process()
A process()


Change the order in the inheritance declaration and the MRO changes accordingly.

In [49]:
class D(C, B):
    pass

print(D.__mro__)

(<class '__main__.D'>, <class '__main__.C'>, <class '__main__.B'>, <class '__main__.A'>, <class 'object'>)


### Example: multiple level inheritance

In [50]:
O = object
class F(O): pass
class E(O): pass
class D(O): pass
class C(D, F): pass
class B(D, E): pass
class A(B, C): pass

In [51]:
print(A.__mro__)

(<class '__main__.A'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.D'>, <class '__main__.E'>, <class '__main__.F'>, <class 'object'>)


In [52]:
print(B.__mro__)

(<class '__main__.B'>, <class '__main__.D'>, <class '__main__.E'>, <class 'object'>)


In [53]:
print(C.__mro__)

(<class '__main__.C'>, <class '__main__.D'>, <class '__main__.F'>, <class 'object'>)


In [54]:
print(D.__mro__)
print(E.__mro__)
print(F.__mro__)

(<class '__main__.D'>, <class 'object'>)
(<class '__main__.E'>, <class 'object'>)
(<class '__main__.F'>, <class 'object'>)


In [55]:
a = A()
print('a {} A'.format('is' if isinstance(a, A) else 'is not'))
print('a {} B'.format('is' if isinstance(a, B) else 'is not'))
print('a {} C'.format('is' if isinstance(a, C) else 'is not'))
print('a {} D'.format('is' if isinstance(a, D) else 'is not'))
print('a {} E'.format('is' if isinstance(a, E) else 'is not'))
print('a {} F'.format('is' if isinstance(a, F) else 'is not'))

a is A
a is B
a is C
a is D
a is E
a is F


## Abstract base class (abc)

Python [abstract base class (abc)](https://docs.python.org/3/library/abc.html) provides the capabilities to overload `isinstance()` and `issubclass()`, and define abstract methods.

We can use `register()` method to ask a class `MyABC` that is not in a inheritance chain of another class `A` to be a 'virtual' base class of the latter.

In [56]:
import abc

class MyABC(metaclass=abc.ABCMeta):
    pass

As we know, `A` is not a subclass of `MyABC`:

In [57]:
print('A {} a subclass of MyABC'.format('is' if issubclass(A, MyABC) else 'is not'))

A is not a subclass of MyABC


But once we register `MyABC` to be a virtual base class of `A`, we will see `A` is a subclass of `MyABC`:

In [58]:
MyABC.register(A)

print('A {} a subclass of MyABC'.format('is' if issubclass(A, MyABC) else 'is not'))

A is a subclass of MyABC


## Abstract method

Using abc, we can add abstract methods to an class (making it abstract).

In [59]:
import abc

class AbstractClass(metaclass=abc.ABCMeta):

    @abc.abstractmethod
    def process(self):
        pass

An abstract class cannot be instantiated:

In [60]:
try:
    a = AbstractClass()
except:
    traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-60-66c429136655>", line 2, in <module>
    a = AbstractClass()
TypeError: Can't instantiate abstract class AbstractClass with abstract methods process


In a derived class, the abstract method needs to be overridden:

In [61]:
class GoodConcreteClass(AbstractClass):

    def process(self):
        print('GoodConcreteClass process')

g = GoodConcreteClass()
g.process()

GoodConcreteClass process


If the abstract method is not overridden, it is a bad derived class and the program does not run:

In [62]:
class BadConcreteClass(AbstractClass):
    pass

try:
    b = BadConcreteClass()
except:
    traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-62-19ed4d2ea3d6>", line 5, in <module>
    b = BadConcreteClass()
TypeError: Can't instantiate abstract class BadConcreteClass with abstract methods process


# Reference

1. K. Barrett, B. Cassels, P. Haahr, D. A. Moon, K. Playford, and P. T. Withington, “A monotonic superclass linearization for Dylan,” SIGPLAN Not., vol. 31, no. 10, pp. 69–82, Oct. 1996, doi: 10.1145/236338.236343.  https://dl.acm.org/doi/10.1145/236338.236343.